# Prospecting LA's Backyard Houses with Machine Learning
  
  
## Anupama Garla  


## Generate Backyard House Lot Database

In [ ]:
import pandas as pd

#get pandas and sql to work together
import psycopg2 as pg
import pandas.io.sql as pd_sql

from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

###  2.0 Pull lot data from pSQL that meets basic parameters of backyard house eligibility

In [17]:
# Connect to psql

params = {
    'host': 'localhost',  # connecting to local version of psql
    'user': 'agar',
    'dbname': 'properties',    # DB connect to
    'port': 5432          # port open 
}

# unpack params dict to connect
connection = pg.connect(**params)

# 1. Join Parcels and Assessors tables on AIN number
# 2. Generate a DataFrame of eligible lots that meet backyard building house code basics:
# area of parcel is at least 920 sf bigger than area of main house
# main house is at least twice the size of a 1-bedroom backyard house - 2*400 sf
# zoning is for Single Family Residence
query_views = """
create view parr
as select cast(parcels."AIN" as bigint), parcels."ShapeSTArea", (parcels."ShapeSTArea" - parcels."SQFTmain1") as "void",
		parcels."ShapeSTLength",  
		parcels."SitusFullAddress", parcels."APN", parcels."CENTER_X", parcels."CENTER_Y" 
from parcels 
where parcels."UseType" = 'Residential' 
	and (parcels."ShapeSTArea" - parcels."SQFTmain1") >=920 
	and (parcels."SQFTmain1" >= 800)  
;

create view ass
as select *
from "Assessors_2019"
where "SpecificUseType" = 'Single Family Residence';
"""

#Generate Backyard House Eligible Lot Database
adus_df = pd.read_sql_query(query, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)
adus_df.head()

,ZIPcode,TaxRateArea_CITY,AIN,RollYear,TaxRateArea,AssessorID,PropertyLocation,PropertyType,PropertyUseCode,GeneralUseType,SpecificUseType,SpecificUseDetail1,SpecificUseDetail2,totBuildingDataLines,YearBuilt,EffectiveYearBuilt,SQFTmain,Bedrooms,Bathrooms,Units,RecordingDate,LandValue,LandBaseYear,ImprovementValue,ImpBaseYear,TotalLandImpValue,HomeownersExemption,RealEstateExemption,FixtureValue,FixtureExemption,PersonalPropertyValue,PersonalPropertyExemption,isTaxableParcel?,TotalValue,TotalExemption,netTaxableValue,SpecialParcelClassification,AdministrativeRegion,Cluster,ParcelBoundaryDescription,HouseNo,HouseFraction,StreetDirection,StreetName,UnitNo,City,ZIPcode5,rowID,CENTER_LAT,CENTER_LON,Location 1,AIN,ShapeSTArea,void,ShapeSTLength,SitusFullAddress,APN,CENTER_X,CENTER_Y
0,90731-2934,LOS ANGELES,7458003016,2019,14,7458-003-016,740 ORO TER LOS ANGELES CA 90731,SFR,0100,Residential,Single Family Residence,None,None,1,1927,1927,1068,2,1,1,20021217,23926.0,1975,15790.0,1975,39716.0,7000.0,0.0,0.0,0.0,0.0,0.0,Y,39716.0,7000.0,32716.0,None,14,14183,VISTA DEL ORO N 37.5 FT OF W 75 FT OF LOT 5...,740,None,None,ORO TER,None,LOS ANGELES CA,90731.0,20197458003016,33.737149,-118.301649,"(33.73714897, -118.30164891)",7458003016,2809.644531,1741.644531,224.835765,740 ORO TER LOS ANGELES CA 90731,7458-003-016,6.469953e+06,1.726860e+06
1,90049-1610,LOS ANGELES,4368011041,2019,67,4368-011-041,1070 CASIANO RD LOS ANGELES CA 90049,SFR,0101,Residential,Single Family Residence,None,Pool,1,1948,1957,3664,4,5,1,20150108,800994.0,1999,470934.0,1999,1271928.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,1271928.0,0.0,1271928.0,None,07,7199,TR=11028 SEE ASSESSOR'S MAPS POR OF LOT 41 BLK 3,1070,None,None,CASIANO RD,None,LOS ANGELES CA,90049.0,20194368011041,34.079679,-118.469282,"(34.07967865, -118.46928191)",4368011041,41704.123047,38040.123047,1029.069846,1070 CASIANO RD LOS ANGELES CA 90049,4368-011-041,6.419566e+06,1.851709e+06
2,90094,LOS ANGELES,4211032193,2019,1515,4211-032-193,"13045 W PACIFIC PROMENADE, 318 PLAYA VISTA CA...",CND,010C,Residential,Single Family Residence,None,Condominium,1,2007,2007,1160,2,2,1,20140228,398354.0,2014,288464.0,2014,686818.0,7000.0,0.0,0.0,0.0,0.0,0.0,Y,686818.0,7000.0,679818.0,None,09,9496,TR=49104-01 LOT 10 PH 3 CONDO UNIT 318 (AIRSPA...,13045,None,W,PACIFIC PROMENADE,318,PLAYA VISTA CA,90094.0,20194211032193,33.972902,-118.424457,"(33.97290243, -118.42445729)",4211032193,71339.170898,70179.170898,1201.571183,13045 W PACIFIC PROMENADE 318 PLAYA VISTA CA 9...,4211-032-193,6.432976e+06,1.812791e+06
3,91307-1408,LOS ANGELES,2028019016,2019,16,2028-019-016,23445 BALMORAL LN LOS ANGELES CA 91307,SFR,0101,Residential,Single Family Residence,None,Pool,1,1961,1961,1842,4,2,1,20160712,323779.0,2001,97654.0,2001,421433.0,7000.0,0.0,0.0,0.0,0.0,0.0,Y,421433.0,7000.0,414433.0,None,02,2145,TRACT # 25870 LOT 97,23445,None,None,BALMORAL LN,None,LOS ANGELES CA,91307.0,20192028019016,34.197978,-118.637806,"(34.1979776, -118.63780609)",2028019016,29901.567383,28059.567383,702.805137,23445 BALMORAL LN LOS ANGELES CA 91307,2028-019-016,6.368809e+06,1.895040e+06
4,91040-3355,LOS ANGELES,2559026010,2019,13,2559-026-010,7827 MCGROARTY ST LOS ANGELES CA 91040,SFR,0100,Residential,Single Family Residence,None,None,1,1956,1956,1480,3,2,1,20020417,222478.0,2002,121830.0,2002,344308.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,344308.0,0.0,344308.0,None,03,3177,TRACT # 22518 LOT 27,7827,None,None,MCGROARTY ST,None,LOS ANGELES CA,91040.0,20192559026010,34.252607,-118.303697,"(34.25260673, -118.3036971)",2559026010,6648.419922,5168.419922,327.302313,7827 MCGROARTY ST LOS ANGELES CA 91040,2559-026-010,6.469897e+06,1.914447e+06


### Examine features to see which ones to use for backyard house calcs

In [21]:
# Look closer at the columns to see what we need
query_small = """
select * from ass
inner JOIN parr
on parr."AIN" = ass."AIN"
limit 3;
"""

adus_sm_df = pd.read_sql_query(query_small, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)
adus_sm_df.transpose()

,0,1,2
ZIPcode,91790-5433,90066-3519,91107-5211
TaxRateArea_CITY,unincorporated,LOS ANGELES,unincorporated
AIN,8490005038,4245018041,5331002047
RollYear,2019,2019,2019
TaxRateArea,9517,67,7622
AssessorID,8490-005-038,4245-018-041,5331-002-047
PropertyLocation,1814 WREN AVE WEST COVINA CA 91790,12911 FERNDALE AVE LOS ANGELES CA 90066,2560 SAN PASQUAL ST PASADENA CA 91107
PropertyType,SFR,SFR,SFR
PropertyUseCode,0100,0100,0100
GeneralUseType,Residential,Residential,Residential


### 2.1 Reduce features to those essential for backyard house calculations and price modeling

In [27]:
adus_calc_df = adus_df[['ShapeSTArea', 'void', 'SQFTmain', 'SitusFullAddress', 'AIN', 
                   'ShapeSTLength', 'APN', 'CENTER_LAT',
       'CENTER_LON', 'Location 1']]

adus_calc_df.head()

,ShapeSTArea,void,SQFTmain,SitusFullAddress,AIN,ShapeSTLength,APN,CENTER_LAT,CENTER_LON,Location 1
0,2809.644531,1741.644531,1068,740 ORO TER LOS ANGELES CA 90731,7458003016,224.835765,7458-003-016,33.737149,-118.301649,"(33.73714897, -118.30164891)"
1,41704.123047,38040.123047,3664,1070 CASIANO RD LOS ANGELES CA 90049,4368011041,1029.069846,4368-011-041,34.079679,-118.469282,"(34.07967865, -118.46928191)"
2,71339.170898,70179.170898,1160,13045 W PACIFIC PROMENADE 318 PLAYA VISTA CA 9...,4211032193,1201.571183,4211-032-193,33.972902,-118.424457,"(33.97290243, -118.42445729)"
3,29901.567383,28059.567383,1842,23445 BALMORAL LN LOS ANGELES CA 91307,2028019016,702.805137,2028-019-016,34.197978,-118.637806,"(34.1979776, -118.63780609)"
4,6648.419922,5168.419922,1480,7827 MCGROARTY ST LOS ANGELES CA 91040,2559026010,327.302313,2559-026-010,34.252607,-118.303697,"(34.25260673, -118.3036971)"


### 2.2 Determine which lots can accomodate 1-3 bedroom backyard houses

In [29]:
#apply bedroom capacity function to Lot DataFrame

adus_calc_df["adu_br"] =  adus_calc_df.apply(lambda x: bedroom_count(x.SQFTmain, x.ShapeSTArea), axis=1)

/Users/agar/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
#Drop 0 bedrooom lots - I don't support building studios!
adus_bro = adus_calc_df[adus_calc_df.adu_br != 0]
adus_bro.head()

,Unnamed: 0,ShapeSTArea,void,SQFTmain,SitusFullAddress,AIN,ShapeSTLength,APN,CENTER_X,CENTER_Y,adu_br
1,1,41704.123047,38040.123047,3664,1070 CASIANO RD LOS ANGELES CA 90049,4368011041,1029.069846,4368-011-041,6.419566e+06,1.851709e+06,3
2,2,71339.170898,70179.170898,1160,13045 W PACIFIC PROMENADE 318 PLAYA VISTA CA 9...,4211032193,1201.571183,4211-032-193,6.432976e+06,1.812791e+06,1
3,3,29901.567383,28059.567383,1842,23445 BALMORAL LN LOS ANGELES CA 91307,2028019016,702.805137,2028-019-016,6.368809e+06,1.895040e+06,2
4,4,6648.419922,5168.419922,1480,7827 MCGROARTY ST LOS ANGELES CA 91040,2559026010,327.302313,2559-026-010,6.469897e+06,1.914447e+06,2
5,5,7650.847656,6556.847656,1094,3918 W CHANDLER BLVD BURBANK CA 91505,2476023001,353.532450,2476-023-001,6.455404e+06,1.884487e+06,1


In [10]:
adus_bro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1597449 entries, 1 to 1689523
Data columns (total 11 columns):
Unnamed: 0          1597449 non-null int64
ShapeSTArea         1597449 non-null float64
void                1597449 non-null float64
SQFTmain            1597449 non-null int64
SitusFullAddress    1596354 non-null object
AIN                 1597449 non-null int64
ShapeSTLength       1597449 non-null float64
APN                 1597449 non-null object
CENTER_X            1597449 non-null float64
CENTER_Y            1597449 non-null float64
adu_br              1597449 non-null int64
dtypes: float64(5), int64(4), object(2)
memory usage: 146.3+ MB


### 2.3 Save out Backyard House Database 

In [11]:
#Save Final Backyard House Database with number of bedrooms acommodated
adus_bro.to_csv("adu_bedrooms_trimmed.csv")

In [13]:
# Push Backyard House Database to sql
connection_string = f'postgres://agar:{params["host"]}@{params["host"]}:{params["port"]}/properties'
engine = create_engine(connection_string, pool_pre_ping=True)

adus_bro.iloc[:0].to_sql("adu_size", engine, index=False)
adus_bro.iloc[:].to_sql("adu_size", engine, index=False, if_exists = 'append', chunksize = 1000)